In [17]:
import pandas as pd
import Property_Data
from math import cos, asin, sqrt

In [18]:
def distance(lat1, lon1, lat2, lon2):
    p = 0.017453292519943295
    a = 0.5 - cos((lat2 - lat1) * p) / 2 + cos(lat1 * p) * \
        cos(lat2 * p) * (1 - cos((lon2 - lon1) * p)) / 2
    return 12742 * asin(sqrt(a)) # in kms

In [19]:
PERCENTAGE_VALID_LOW = 40
PERCENTAGE_VALID_HIGH = 100

DISTANCE_WEIGHT = 0.3
BUDGET_WEIGHT = 0.3
BATHROOM_WEIGHT = 0.2
BEDROOM_WEIGHT = 0.2

DISTANCE_THRESHOLD_LOW = 2
DISTANCE_THRESHOLD_HIGH = 10

# considering the avg of budget max-min to find the percentage
BUDGET_THRESHOLD_LOW = 10
BUDGET_THRESHOLD_HIGH = 25

ROOMS_THRESHOLD_LOW = 0
ROOMS_THRESHOLD_HIGH = 2

In [20]:
def getDistanceMatch(req_data, present_data):
    lat1 = req_data['lat']
    lon1 = req_data['lon']
    lat2 = present_data['lat']
    lon2 = present_data['lon']
    old_value = distance(lat1, lon1, lat2, lon2)
    old_max = DISTANCE_THRESHOLD_HIGH
    old_min = DISTANCE_THRESHOLD_LOW
    if 0 <= old_value <= old_min:
        return 100
    if old_min < old_value < old_max:
        new_max = PERCENTAGE_VALID_HIGH
        new_min = PERCENTAGE_VALID_LOW
        old_range = (old_max - old_min)
        new_range = (new_max - new_min)
        new_value = (((old_value - old_min) * new_range) / old_range) + new_min
        return 140 - new_value
    return 0

In [21]:
def getBudgetMatch(req_data, present_data):
    budget_max = req_data['maxBudget']
    budget_min = req_data['minBudget']
    if (budget_max is None or budget_max is '') and (budget_min is None or budget_min is ''):
        return 0
    if budget_max is None or budget_max is '':
        budget_max = budget_min
    if budget_min is None or budget_min is '':
        budget_min = budget_max
    budget_max = float(budget_max)
    budget_min = float(budget_min)
    avg_budget = (budget_max + budget_min) / 2.0
    old_value = float(present_data['price'])
    old_low_min = budget_min - (avg_budget * BUDGET_THRESHOLD_LOW) / 100
    old_low_min = old_low_min if old_low_min > 0 else 0
    old_low_max = budget_max + (avg_budget * BUDGET_THRESHOLD_LOW) / 100
    old_low_max = old_low_max if old_low_max > 0 else 0
    if old_low_min <= old_value <= old_low_max:
        return 100

    if old_value >= old_low_max:
        old_min = old_low_max
        old_max = budget_max + (avg_budget * BUDGET_THRESHOLD_HIGH) / 100
        old_max = old_max if old_max > 0 else 0
        if old_value <= old_max:
            new_max = PERCENTAGE_VALID_HIGH
            new_min = PERCENTAGE_VALID_LOW
            old_range = (old_max - old_min)
            new_range = (new_max - new_min)
            new_value = (((old_value - old_min) * new_range) /
                         old_range) + new_min
            return 140 - new_value
    else:
        old_max = old_low_min
        old_min = budget_min - (avg_budget * BUDGET_THRESHOLD_HIGH) / 100
        old_min = old_min if old_min > 0 else 0
        if old_value >= old_min:
            new_max = PERCENTAGE_VALID_HIGH
            new_min = PERCENTAGE_VALID_LOW
            old_range = (old_max - old_min)
            new_range = (new_max - new_min)
            new_value = (((old_value - old_min) * new_range) /
                         old_range) + new_min
            return new_value
    return 0

In [22]:
def getBedroomMatch(req_data, present_data):
    bedroom_max = req_data['maxBedrooms']
    bedroom_min = req_data['minBedrooms']
    if (bedroom_max is None or bedroom_max is '') and (bedroom_min is None or bedroom_min is ''):
        return 0
    if bedroom_max is None or bedroom_max is '':
        bedroom_max = bedroom_min
    if bedroom_min is None or bedroom_min is '':
        bedroom_min = bedroom_max
    bedroom_max = int(bedroom_max)
    bedroom_min = int(bedroom_min)
    old_value = int(present_data['bedrooms'])
    old_low_min = bedroom_min - ROOMS_THRESHOLD_LOW
    old_low_min = old_low_min if old_low_min > 0 else 0
    old_low_max = bedroom_max + ROOMS_THRESHOLD_LOW
    old_low_max = old_low_max if old_low_max > 0 else 0
    if old_low_min <= old_value <= old_low_max:
        return 100

    if old_value >= old_low_max:
        old_min = old_low_max
        old_max = bedroom_max + ROOMS_THRESHOLD_HIGH
        old_max = old_max if old_max > 0 else 0
        if old_value <= old_max:
            new_max = PERCENTAGE_VALID_HIGH
            new_min = PERCENTAGE_VALID_LOW
            old_range = (old_max - old_min)
            new_range = (new_max - new_min)
            new_value = (((old_value - old_min) * new_range) /
                         old_range) + new_min
            return 140 - new_value
    else:
        old_max = old_low_min
        old_min = bedroom_min - ROOMS_THRESHOLD_HIGH
        old_min = old_min if old_min > 0 else 0
        if old_value >= old_min:
            new_max = PERCENTAGE_VALID_HIGH
            new_min = PERCENTAGE_VALID_LOW
            old_range = (old_max - old_min)
            new_range = (new_max - new_min)
            new_value = (((old_value - old_min) * new_range) /
                         old_range) + new_min
            return new_value
    return 0

In [23]:
def getBathroomMatch(req_data, present_data):
    bedroom_max = req_data['maxBathrooms']
    bedroom_min = req_data['minBathrooms']
    if (bedroom_max is None or bedroom_max is '') and (bedroom_min is None or bedroom_min is ''):
        return 0
    if bedroom_max is None or bedroom_max is '':
        bedroom_max = bedroom_min
    if bedroom_min is None or bedroom_min is '':
        bedroom_min = bedroom_max
    bedroom_max = int(bedroom_max)
    bedroom_min = int(bedroom_min)
    old_value = int(present_data['bathrooms'])
    old_low_min = bedroom_min - ROOMS_THRESHOLD_LOW
    old_low_min = old_low_min if old_low_min > 0 else 0
    old_low_max = bedroom_max + ROOMS_THRESHOLD_LOW
    old_low_max = old_low_max if old_low_max > 0 else 0
    if old_low_min <= old_value <= old_low_max:
        return 100

    if old_value >= old_low_max:
        old_min = old_low_max
        old_max = bedroom_max + ROOMS_THRESHOLD_HIGH
        old_max = old_max if old_max > 0 else 0
        if old_value <= old_max:
            new_max = PERCENTAGE_VALID_HIGH
            new_min = PERCENTAGE_VALID_LOW
            old_range = (old_max - old_min)
            new_range = (new_max - new_min)
            new_value = (((old_value - old_min) * new_range) /
                         old_range) + new_min
            return 140 - new_value
    else:
        old_max = old_low_min
        old_min = bedroom_min - ROOMS_THRESHOLD_HIGH
        old_min = old_min if old_min > 0 else 0
        if old_value >= old_min:
            new_max = PERCENTAGE_VALID_HIGH
            new_min = PERCENTAGE_VALID_LOW
            old_range = (old_max - old_min)
            new_range = (new_max - new_min)
            new_value = (((old_value - old_min) * new_range) /
                         old_range) + new_min
            return new_value
    return 0

In [24]:
def getMatch(req_data, app_data):
    distance_match = getDistanceMatch(req_data, app_data)
    budget_match = getBudgetMatch(req_data, app_data)
    bedroom_match = getBedroomMatch(req_data, app_data)
    bathroom_match = getBathroomMatch(req_data, app_data)
    return round(
        distance_match * DISTANCE_WEIGHT
        + budget_match * BUDGET_WEIGHT
        + bedroom_match * BEDROOM_WEIGHT
        + bathroom_match * BATHROOM_WEIGHT,
        2)

In [25]:
app_db = Property_Data.DATA

In [26]:
def getTopMatches(req_data):
    ans = []
    count = 10
    for data in app_db:
        if count < 0:
            break
        count = count - 1
        match = getMatch(req_data, data)
        if match >= 40:
            obj = data.copy()
            obj.update({'match': match})
            ans.append(obj)
    ans.sort(key=lambda k: k['match'], reverse=True)
    return ans

# Sample Input for testing the results

##### Minimum Budget = 8305.39
##### Maximum Budget = 18305.39
##### Latitude = 18.37
##### Longitude = 121.511
##### Minimum number of bedrooms = 1
##### Maximum number of bedrooms = 4
##### Minimum number of bathrooms = 1
##### Maximum number of bathrooms = 4

###### .

In [27]:
input1={"lat": 18.3721392,"lon": 121.5111211,"minBedrooms": 1,"maxBedrooms": 2,"minBathrooms": 1, "maxBathrooms": 4,"minBudget": "8305.39","maxBudget": "18305.39"}

# Results

In [28]:
df=pd.DataFrame.from_dict(data=getTopMatches(input1))
df

,bathrooms,bedrooms,email,id,lat,lon,match,name,price
0,2,2,mheaviside0@symantec.com,1,18.372139,121.511128,100.0,Jerde Group,8305.39
1,2,2,ddybbe6@dell.com,7,59.917331,10.818031,70.0,Altenwerth Inc,8435.96
2,3,3,opharrow1@time.com,2,45.262508,17.427272,64.0,Murray-Schaden,8911.71
3,4,4,abadrick2@nba.com,3,-7.325472,108.207356,58.0,Mante Inc,7741.58
4,5,5,snorthover3@baidu.com,4,14.674129,120.511291,44.0,"Bruen, Goldner and Bednar",7171.20
5,5,5,esigfrid9@shinystat.com,10,56.578971,85.102674,44.0,"Walter, Herman and Murray",7239.76
6,1,1,bcampo5@yahoo.com,6,12.564028,123.730411,40.0,Auer Group,1526.33
